# WZE-UAV Image Classification using Deep Learning

In [1]:
import os
import glob
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

In [2]:
import torch
import torchvision.transforms as T
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torchmetrics import *
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from mlxtend.plotting import plot_confusion_matrix

In [3]:
import wze_uav.data_loader as data_loader
import wze_uav.visualization as visualization
import wze_uav.models as models
from wze_uav.engine import *
from wze_uav.utils2 import *
from wze_uav.log_writer import create_writer
from wze_uav.datasplit import *
from efficientnet import model #custom effnet

#### Get PyTorch version

In [4]:
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.1+cu116
torchvision version: 0.14.1+cu116


#### Preparing device agnostic code 

In [5]:
# ensure device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# get index of currently selected device
print(f"Index of current divice: {torch.cuda.current_device()}")
# get number of GPUs available
print(f"Number of GPUs available: {torch.cuda.device_count()}")
# get the name of the device
print(f"GPU Model: {torch.cuda.get_device_name(0)}")

cuda
Index of current divice: 0
Number of GPUs available: 1
GPU Model: Quadro RTX 8000


#### Ensure reproducibility 

In [6]:
# for more information, see also: https://pytorch.org/docs/stable/notes/randomness.html

# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)
    # seed for numpy
    np.random.seed(seed)

set_seeds() 

# Set to true -> might speed up the process but should be set to False if reproducible results are desired
torch.backends.cudnn.benchmark = False


#### Define file directory

In [7]:
#####################################################################################
# 3 channel input (r-g-b)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\rgb"

# 4 channel input (r-g-b-nir)
data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\r-g-b-nir"

# 5 channel input (r-g-b-re-nir)
#data_path = r"D:\Drohnendaten\10_WZE-UAV\Auswertung_findatree\Datasplit\ROI\r-g-b-re-nir"
#####################################################################################

#### Get all file paths

In [8]:
fn_list = os.listdir(data_path)
path_list = []
# Iterate over all datafiles
for year in fn_list:
    year_dir = f'{data_path}\\{year}'
    for filename in os.listdir(year_dir):
        path = f'{year_dir}\\{filename}'
        path_list.append(path)

#### Create unique hash IDs for every individual tree

In [9]:
hashID_dict = data_loader.get_unique_treeID(path_list)

Creating unique tree IDs...:   0%|          | 0/647 [00:00<?, ?it/s]

#### Import all imagery, labels and other features from hdf5 files

In [10]:
image_set, label_set, species_set, kkl_set, bk_set, hash_id = data_loader.hdf5_to_img_label_v2(path_list,
                                                                                               hashID_dict,
                                                                                               load_sets=["images_masked"])

Processing hdf5 datasets:   0%|          | 0/647 [00:00<?, ?it/s]

#### Convert nbv to classes

In [11]:
label_set = nbv_to_sst_3classes(label_set)

#### Split data and seperate a test dataset

In [12]:
image_set, label_set, hash_id, species_set, test_image_set, test_label_set, test_hash_id, test_species_set = data_split(image_set, label_set, hash_id, species_set, test_size=0.15, random_state=42)


There are 7087 unique values within hash_id.
Check shapes:

Images train dataset: (15973, 250, 250, 3)
Labels train dataset: (15973, 1)

Images test dataset: (2832, 250, 250, 3)
Labels test dataset: (2832, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 14465
Stressed trees in train dataset: 1240
Dead trees in train dataset: 268
Healthy trees in test dataset: 2551
Stressed trees in test dataset: 237
Dead trees in test dataset: 44
Ratio health trees in test dataset: 0.1499177244945933
Ratio stressed trees in test dataset: 0.16046039268788084
Ratio dead trees in test dataset: 0.14102564102564102


#### Check if any hash ID is in both train and test dataset

In [13]:
hash_set = set(hash_id.flatten())
test_hash_set = set(test_hash_id.flatten())
intersection = hash_set.intersection(test_hash_set)
if intersection:
    print(f"Hash_id values in both train and test sets: {intersection}")
else:
    print("There are no hash_id values in both train and test datasets. The datasplit was successful")

There are no hash_id values in both train and test datasets. The datasplit was successful


#### Check feature distribution of the Test dataset

In [14]:
def count_occurrences(data, value):
    count = 0
    for item in data:
        if item == value:
            count += 1
    return count

print("Test dataset")
print(f"Test data healthy trees: {count_occurrences(test_label_set, 0)}")
print(f"Test data stressed trees: {count_occurrences(test_label_set, 1)}")
print(f"Test data dead trees: {count_occurrences(test_label_set, 2)}")
print(f"Test data pine trees: {count_occurrences(test_species_set, 134)}")
print(f"Test data spruces: {count_occurrences(test_species_set, 118)}")
print("-"*50)

print("Remaining dataset")
print(f"Remaining data healthy trees: {count_occurrences(label_set, 0)}")
print(f"Remaining data stressed trees: {count_occurrences(label_set, 1)}")
print(f"Remaining data dead trees: {count_occurrences(label_set, 2)}")
print(f"Remaining data pine trees: {count_occurrences(species_set, 134)}")
print(f"Remaining data spruces: {count_occurrences(species_set, 118)}")
print("-"*50)

Test dataset
Test data healthy trees: 2551
Test data stressed trees: 237
Test data dead trees: 44
Test data pine trees: 755
Test data spruces: 1041
--------------------------------------------------
Remaining dataset
Remaining data healthy trees: 14465
Remaining data stressed trees: 1240
Remaining data dead trees: 268
Remaining data pine trees: 4036
Remaining data spruces: 5942
--------------------------------------------------


In [15]:
# train transform with augmentation. 
transform_train = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(p=0.5), transforms.RandomVerticalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=[0,360])])

# test and val dataset transform without augmentation. 
transform = transforms.Compose([transforms.ToTensor()])

# class names need to fit the customDataset class used e.g. 3 classes -> use CustomDataset3Classes
#class_names = ['healthy', 'slightly_stressed', 'moderately_stressed', 'highly_stressed', 'dead']
#class_names = ['healthy', 'moderately_stressed', 'highly_stressed', 'dead']
class_names = ['healthy', 'stressed', 'dead']

# set seeds
g = torch.Generator()
g.manual_seed(42)
NUM_WORKERS=3 # should be changed, depending on the system used
batch_size=32

#### Define variables and parameters

In [16]:
# 1. Define number of epochs
epochs = 50
n_bands = image_set[0].shape[2] # get number of bands

# 2. Define model
num_classes = len(class_names)
unfreeze = True # all layer weights get updated
dropout_rate = 0.5 #define dropout rate
model_name = "EffNet_b7"

# 3. Define loss, optimizer and learning rate scheduler
lr = 0.0011 # define learning rate
gamma = 0.85 # how fast the learning rate decreases per epoch (low number=faster decrease)
patience = 15
#step_size = 5

# 4. Create target folder name were to save the tensorboard event files
target_dir = "EffNet_kFold_RGBNIR_3classes_log_evaluation"
experiment_name = 'kFold_RGBNIR_3classes'

#----------------------------------------------------------------------
torch.cuda.empty_cache()
print(f"Memory allocated: {torch.cuda.memory_allocated()} bytes") 





Memory allocated: 0 bytes


#### Run k-Fold cross-validation on EfficientNet

In [ ]:
%%time

# Set the random seeds
set_seeds(42)

# Define the number of folds
num_folds = 5

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

unique_values = np.unique(hash_id) # now there are less unique values left because we seperated the test dataset
num_unique = len(unique_values)
print(f"There are {num_unique} unique values within hash_id.\n")

# Loop over each fold
for fold, (train_ids, val_ids) in enumerate(kf.split(unique_values)):
    torch.cuda.empty_cache()
    
    # 1. Split data into train and validation set
    # Get the training and testing data for this fold
    # Use np.isin() to create boolean arrays indicating which indices belong to train or test sets
    train_indices = np.isin(hash_id, unique_values[train_ids])
    val_indices = np.isin(hash_id, unique_values[val_ids])
    
    # Reshape boolean arrays to match shape of image_set and label_set
    train_indices = train_indices.reshape(-1, 1)
    val_indices = val_indices.reshape(-1, 1)
    
    # Select images and labels for train and validation sets
    train_image_set = image_set[train_indices[:, 0]]
    train_label_set = label_set[train_indices[:, 0]]
    train_hash_id = hash_id[train_indices[:, 0]]
    train_species_set = species_set[train_indices[:, 0]]
    val_image_set = image_set[val_indices[:, 0]]
    val_label_set = label_set[val_indices[:, 0]]
    val_hash_id = hash_id[val_indices[:, 0]]
    val_species_set = species_set[val_indices[:, 0]]
    train_label_set = train_label_set.reshape(-1, 1)
    val_label_set = val_label_set.reshape(-1, 1)
    train_hash_id = train_hash_id.reshape(-1, 1)
    val_hash_id = val_hash_id.reshape(-1, 1)
    train_species_set = train_species_set.reshape(-1, 1)
    val_species_set = val_species_set.reshape(-1, 1)
            
    print("Check shapes:\n")
    print(f"Images train dataset: {train_image_set.shape}")
    print(f"Labels train dataset: {train_label_set.shape}\n")
    
    print(f"Images validation dataset: {val_image_set.shape}")
    print(f"Labels validation dataset: {val_label_set.shape}\n")
    print('-'*50)
    print (f"Check if the split was stratified: (random_state=42)")
    print(f"Healthy trees in train dataset: {np.count_nonzero(train_label_set == 0)}")
    print(f"Stressed trees in train dataset: {np.count_nonzero(train_label_set == 1)}")
    print(f"Dead trees in train dataset: {np.count_nonzero(train_label_set == 2)}")
    print(f"Healthy trees in validation dataset: {np.count_nonzero(val_label_set == 0)}")
    print(f"Stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)}")
    print(f"Dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)}")
    print(f"Ratio health trees in validation dataset: {np.count_nonzero(val_label_set == 0)/np.count_nonzero(label_set == 0)}")
    print(f"Ratio stressed trees in validation dataset: {np.count_nonzero(val_label_set == 1)/np.count_nonzero(label_set == 1)}")
    print(f"Ratio dead trees in validation dataset: {np.count_nonzero(val_label_set == 2)/np.count_nonzero(label_set == 2)}")
    print("-"*50)
    
    # 2. Create train and validation dataset. (choose custom dataset loader with 3 - 5 classes)
    print(f"\nCreating datasets for fold: {fold + 1}\n")
    train_dataset = data_loader.CustomDataset3Classes_v2(data=train_image_set, labels=train_label_set, class_names=class_names, species = train_species_set,
                                                         transform=transform_train)
    
    val_dataset = data_loader.CustomDataset3Classes_v2(data=val_image_set, labels=val_label_set, class_names=class_names,
                                                       species = val_species_set, transform=transform)
    
    # 3. Create train and validation dataloader
    # create sampler for oversampling of the minority classes
    sampler = data_loader.data_sampler(dataset=train_dataset, class_names=class_names)
    print(f"Creating dataloaders for fold: {fold +1}\n")
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, generator=g,
                              sampler=sampler, shuffle=False, drop_last=True) # shuffle false because of the sampler

    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, persistent_workers=True, pin_memory=True, num_workers=NUM_WORKERS, shuffle=False,
                             drop_last=True)
    
    model_custom = model.EfficientNet.from_pretrained('efficientnet-b7', in_channels=n_bands, num_classes=num_classes, dropout_rate=dropout_rate)
    model_custom.to(device) # putting model to device
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)
    #lr_scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    lr_scheduler = ExponentialLR(optimizer, gamma=gamma)
    
    fold += 1
    print(f"\n[INFO] Fold number: {fold}")
    print(f"[INFO] Number of epochs: {epochs}")
    print(f"[INFO] Batch_size: {batch_size}")
    print(f"[INFO] Number of bands: {n_bands}")
    print(f"[INFO] Dropout rate: {dropout_rate}")
    print(f"[INFO] Gamma learning rate: {gamma}")
    # 4. Train model with k fold dataloaders and track experiments
    if fold == 1:
        fold1_results = train(model=model_custom, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=create_writer(target_dir=target_dir, experiment_name=experiment_name, model_name=model_name, extra=f"{epochs}_epochs"), early_stop_patience = patience)
    elif fold == 2:
        fold2_results = train(model=model_custom, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=create_writer(target_dir=target_dir, experiment_name=experiment_name, model_name=model_name, extra=f"{epochs}_epochs"), early_stop_patience = patience)
    elif fold == 3:
        fold3_results = train(model=model_custom, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=create_writer(target_dir=target_dir, experiment_name=experiment_name, model_name=model_name, extra=f"{epochs}_epochs"), early_stop_patience = patience)
    elif fold == 4:
        fold4_results = train(model=model_custom, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=create_writer(target_dir=target_dir, experiment_name=experiment_name, model_name=model_name, extra=f"{epochs}_epochs"), early_stop_patience = patience)
    else:
        fold5_results = train(model=model_custom, model_name=model_name, n_bands=n_bands, batch_size=batch_size,train_dataloader=train_dataloader, val_dataloader=val_dataloader, 
                        optimizer=optimizer, loss_fn=loss_fn, lr_scheduler=lr_scheduler, num_classes=num_classes, epochs=epochs, experiment_num=fold, device=device,
                        writer=create_writer(target_dir=target_dir, experiment_name=experiment_name, model_name=model_name, extra=f"{epochs}_epochs"), early_stop_patience = patience)
    
    del train_indices, val_indices, train_image_set, train_label_set, train_hash_id, train_species_set, val_image_set, val_label_set, val_hash_id, val_species_set,
    train_dataset, val_dataset, sampler, train_dataloader, val_dataloader, model_custom, loss_fn, optimizer, lr_scheduler
    print("-"*50 + "\n")

There are 6023 unique values within hash_id.

Check shapes:

Images train dataset: (12770, 250, 250, 3)
Labels train dataset: (12770, 1)

Images validation dataset: (3203, 250, 250, 3)
Labels validation dataset: (3203, 1)

--------------------------------------------------
Check if the split was stratified: (random_state=42)
Healthy trees in train dataset: 11550
Stressed trees in train dataset: 983
Dead trees in train dataset: 237
Healthy trees in validation dataset: 2915
Stressed trees in validation dataset: 257
Dead trees in validation dataset: 31
Ratio health trees in validation dataset: 0.20152091254752852
Ratio stressed trees in validation dataset: 0.20725806451612902
Ratio dead trees in validation dataset: 0.11567164179104478
--------------------------------------------------

Creating datasets for fold: 1

Creating dataloaders for fold: 1

[INFO] Created new effnet_b7 model.

[INFO] Fold number: 1
[INFO] Number of epochs: 50
[INFO] Batch_size: 32
[INFO] Number of bands: 3
[INFO]

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.0011
Train loss: 0.6712 | Train precision: 0.6925 | Train recall: 0.6929 | Train f1score: 0.6927 | Train acc: 0.6750 | Train kappa: 0.4450 
Val loss: 0.3040 | Val precision: 0.5366 | Val recall: 0.6396 | Val f1score: 0.5179 | Val acc: 0.9072 | Val kappa: 0.2085 

Epoch: 2 
Learning rate: 0.0009350000000000001
Train loss: 0.5748 | Train precision: 0.7563 | Train recall: 0.7573 | Train f1score: 0.7567 | Train acc: 0.7274 | Train kappa: 0.5358 
Val loss: 0.3409 | Val precision: 0.5739 | Val recall: 0.7804 | Val f1score: 0.6395 | Val acc: 0.8428 | Val kappa: 0.3251 

Epoch: 3 
Learning rate: 0.00079475
Train loss: 0.5448 | Train precision: 0.7777 | Train recall: 0.7736 | Train f1score: 0.7756 | Train acc: 0.7428 | Train kappa: 0.5575 
Val loss: 0.4609 | Val precision: 0.6465 | Val recall: 0.7702 | Val f1score: 0.6615 | Val acc: 0.7600 | Val kappa: 0.2518 

Epoch: 4 
Learning rate: 0.0006755375
Train loss: 0.5234 | Train precision: 0.7918 | Train recall: 0.7863 | 

Epoch: 28 
Learning rate: 1.3667959380042224e-05
Train loss: 0.2393 | Train precision: 0.9229 | Train recall: 0.9261 | Train f1score: 0.9242 | Train acc: 0.9011 | Train kappa: 0.8314 
Val loss: 0.3889 | Val precision: 0.6250 | Val recall: 0.7856 | Val f1score: 0.6815 | Val acc: 0.8506 | Val kappa: 0.3490 

Epoch: 29 
Learning rate: 1.161776547303589e-05
Train loss: 0.2320 | Train precision: 0.9253 | Train recall: 0.9283 | Train f1score: 0.9264 | Train acc: 0.9037 | Train kappa: 0.8362 
Val loss: 0.3816 | Val precision: 0.6248 | Val recall: 0.7850 | Val f1score: 0.6827 | Val acc: 0.8584 | Val kappa: 0.3608 

Epoch: 30 
Learning rate: 9.875100652080506e-06
Train loss: 0.2253 | Train precision: 0.9276 | Train recall: 0.9310 | Train f1score: 0.9289 | Train acc: 0.9079 | Train kappa: 0.8429 
Val loss: 0.3980 | Val precision: 0.6221 | Val recall: 0.7849 | Val f1score: 0.6781 | Val acc: 0.8453 | Val kappa: 0.3401 

Early stopping after epoch 30
------------------------------------------------

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.0011
Train loss: 0.6836 | Train precision: 0.6781 | Train recall: 0.6790 | Train f1score: 0.6785 | Train acc: 0.6674 | Train kappa: 0.4210 
Val loss: 0.3886 | Val precision: 0.7138 | Val recall: 0.5050 | Val f1score: 0.5576 | Val acc: 0.9205 | Val kappa: 0.2958 

Epoch: 2 
Learning rate: 0.0009350000000000001
Train loss: 0.5811 | Train precision: 0.7532 | Train recall: 0.7550 | Train f1score: 0.7540 | Train acc: 0.7315 | Train kappa: 0.5331 
Val loss: 0.3736 | Val precision: 0.6208 | Val recall: 0.7387 | Val f1score: 0.6678 | Val acc: 0.8873 | Val kappa: 0.4139 

Epoch: 3 
Learning rate: 0.00079475
Train loss: 0.5520 | Train precision: 0.7701 | Train recall: 0.7742 | Train f1score: 0.7721 | Train acc: 0.7465 | Train kappa: 0.5550 
Val loss: 0.4140 | Val precision: 0.5964 | Val recall: 0.7584 | Val f1score: 0.6407 | Val acc: 0.8056 | Val kappa: 0.3036 

Epoch: 4 
Learning rate: 0.0006755375
Train loss: 0.5320 | Train precision: 0.7768 | Train recall: 0.7865 | 

Epoch: 28 
Learning rate: 1.3667959380042224e-05
Train loss: 0.2334 | Train precision: 0.9269 | Train recall: 0.9303 | Train f1score: 0.9279 | Train acc: 0.9058 | Train kappa: 0.8366 
Val loss: 0.3683 | Val precision: 0.7134 | Val recall: 0.7928 | Val f1score: 0.7339 | Val acc: 0.8614 | Val kappa: 0.4089 

Epoch: 29 
Learning rate: 1.161776547303589e-05
Train loss: 0.2348 | Train precision: 0.9242 | Train recall: 0.9281 | Train f1score: 0.9256 | Train acc: 0.9044 | Train kappa: 0.8341 
Val loss: 0.3905 | Val precision: 0.7115 | Val recall: 0.7963 | Val f1score: 0.7316 | Val acc: 0.8554 | Val kappa: 0.4016 

Epoch: 30 
Learning rate: 9.875100652080506e-06
Train loss: 0.2324 | Train precision: 0.9243 | Train recall: 0.9312 | Train f1score: 0.9270 | Train acc: 0.9066 | Train kappa: 0.8373 
Val loss: 0.3850 | Val precision: 0.7178 | Val recall: 0.7833 | Val f1score: 0.7295 | Val acc: 0.8510 | Val kappa: 0.3819 

Early stopping after epoch 30
------------------------------------------------

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 
Learning rate: 0.0011
Train loss: 0.6763 | Train precision: 0.6849 | Train recall: 0.6819 | Train f1score: 0.6834 | Train acc: 0.6736 | Train kappa: 0.4355 
Val loss: 0.3048 | Val precision: 0.6988 | Val recall: 0.4977 | Val f1score: 0.5471 | Val acc: 0.9081 | Val kappa: 0.2128 

Epoch: 2 
Learning rate: 0.0009350000000000001
Train loss: 0.5783 | Train precision: 0.7599 | Train recall: 0.7588 | Train f1score: 0.7594 | Train acc: 0.7330 | Train kappa: 0.5392 
Val loss: 0.4768 | Val precision: 0.5519 | Val recall: 0.7531 | Val f1score: 0.6001 | Val acc: 0.7734 | Val kappa: 0.2754 

Epoch: 3 
Learning rate: 0.00079475
Train loss: 0.5338 | Train precision: 0.7838 | Train recall: 0.7852 | Train f1score: 0.7845 | Train acc: 0.7505 | Train kappa: 0.5686 
Val loss: 0.5354 | Val precision: 0.6629 | Val recall: 0.7193 | Val f1score: 0.6405 | Val acc: 0.7484 | Val kappa: 0.2602 

Epoch: 4 
Learning rate: 0.0006755375
Train loss: 0.5207 | Train precision: 0.7898 | Train recall: 0.7865 | 

In [ ]:
# create test dataset
test_dataset = data_loader.CustomDataset3Classes(
    data = test_image_set,
    labels = test_label_set,
    class_names=class_names, 
    species = test_species_set,
    kkl = test_kkl_set,
    transform=transform
)

# create test dataloader
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             persistent_workers=True,
                             pin_memory=True,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             drop_last=True)

In [60]:
# Setup the best model filepath
best_model_path = r"C:\Users\lwfeckesim\01_PyTorch\wze-uav\wze-uav-master\effnet_b0\01_18_epochs.pth"

# Instantiate a new instance of EffNetB0 (to load the saved state_dict() to)
unfreeze=True
best_model = models.create_effnetb0(output_shape=num_classes, unfreeze=unfreeze, dropout_rate=dropout_rate, device=device)

# Load the saved best model state_dict()
best_model.load_state_dict(torch.load(best_model_path))

[INFO] Created new effnet_b0 model.


<All keys matched successfully>

In [ ]:
# get predictions from test dataloader
test_loss, test_precision, test_recall, test_f1_score, test_acc, test_kappa, y_preds, y_labels = make_predictions(model=best_model,
                                 test_dataloader=test_dataloader, 
                                 device=device)

# Setup confusion matrix instance and compare predictions to targets
labels = np.array([0,1,2])
confmat = confusion_matrix(y_labels, y_preds, labels=labels)

# Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat, 
    class_names=class_names, # turn the row and column labels into class names
    figsize=(10, 7)
);

print(f"Test loss: {test_loss}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test F1score: {test_f1_score}")
print(f"Test Kappa: {test_kappa}")
print(f"Test Accuracy: {test_acc}")